<a href="https://colab.research.google.com/github/kankkw/229352-StatisticalLearning/blob/main/Lab05_670510735.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

ปฏิบัติการครั้งที่ 4 กระบวนวิชา 229351 Statistical Learning for Data Science

คำชี้แจง

ให้เริ่มทำปฏิบัติการจาก colab notebook ที่กำหนดให้ จากนั้น share แล้วส่ง link ใน mango.cmu.ac.th

ดาวน์โหลดข้อมูลรถยนต์ชนิดต่างใน link ข้างล่างนี้ หรือรันโค้ดข้างล่าง
https://donlapark.pages.dev/229351/data/Auto.csv  

In [ ]:
!wget https://donlapark.pages.dev/229351/data/Auto.csv

In [ ]:
# import module ที่ต้องใช้
import numpy as np
import pandas as pd
from scipy import stats

In [ ]:
# อ่านไฟล์ csv ก่อนเพื่อหา missing values
auto_df = pd.read_csv('Auto.csv',na_values=["?"])

# ลบแถวที่มี missing values
auto_df = auto_df.dropna()
auto_df.head()

In [ ]:
auto_df.info()

* Predictor: `horsepower` $X = [x_1,x_2,\ldots,x_n]$
* Response: `mpg` $y = [y_1,y_2,\ldots,y_n]$
* สมการ $\hat{y}_i = \hat{\beta}_0+\hat{\beta}_1x_i, \ \ \ \ $     $ i=1,2,\ldots,n$  

In [ ]:
X = auto_df['horsepower']
y = auto_df['mpg']

คำนวณสัมประสิทธิ์ $\hat{\beta}_0,\hat{\beta}_1$ ด้วยสูตร
\begin{align*}
\hat{\beta}_1 &= \frac{\sum_{i=1}^n (x_i-\bar{x})(y_i-\bar{y})}{\sum_{i=1}^n (x_i-\bar{x})^2} \\
&= \frac{\text{Cov}(X,y)}{\text{Var}(X)}\\
\hat{\beta}_0 &= \bar{y}-\hat{\beta}_1\bar{x}
\end{align*}
  
ใช้คำสั่ง $\texttt{np.cov(X,y)}=\begin{pmatrix}
\text{Var}(X)  & \text{Cov}(X,y) \\
\text{Cov}(y,X)  & \text{Var}(y)
\end{pmatrix}
$ สำหรับ covariance matrix ระหว่างเวกเตอร์ $\texttt{X}$ และ $\texttt{y}$  
ใช้คำสั่ง $\texttt{np.mean(X)}$ และ $\texttt{np.mean(y)}$ สำหรับค่าเฉลี่ยของเวกเตอร์ $\texttt{X}$ และ $\texttt{y}$

#### Exercise 1:

- เติมโค้ดในฟังก์ชัน `linear_model` ที่คำนวณค่าสัมประสิทธิ์ (coefficients) จากข้อมูล `X` และ `y`
- เติมโค้ดในฟังก์ชัน `predict` ที่คำนวณค่าทำนาย จากข้อมูลตัวแปรต้น `X` และสัมประสิทธิ์ของโมเดล `beta_0` และ `beta_1`

In [ ]:
def linear_model(X, y):
    x_mean = np.mean(X)
    y_mean = np.mean(y)
    cov = np.cov(X, y, bias=True)[0][1]
    var_x = np.var(X)
    beta_1 = cov / var_x
    beta_0 = y_mean - beta_1 * x_mean
    return beta_0, beta_1

def predict(beta_0, beta_1, X):
    return beta_0 + beta_1 * X

beta_0, beta_1 = linear_model(X, y)
y_pred = predict(beta_0, beta_1, X)

print(beta_0, beta_1)

In [ ]:
beta_0, beta_1 = linear_model(X, y)
y_pred = predict(beta_0 , beta_1, X)

print(beta_0,beta_1)

In [ ]:
# นับจำนวนข้อมูล

print(y.shape)

n = y.shape[0]
print(n)

#### Exercise 2: คำนวณ Residual Sum of Squares (RSS)
$$ \text{RSS} = \sum_{i=1}^n (y_i-\hat{y}_i)^2 $$
โดยใช้คำสั่งต่อไปนี้

$\texttt{np.sum}([x_1,x_2,...,x_n])=x_1+x_2+...+x_n$

$\texttt{np.square}([x_1,x_2,...,x_n])=[x_1^2,x^2_2,...,x^2_n]$

In [ ]:
RSS = np.sum(np.square(y - y_pred))
print("RSS:", RSS)

#### Exercise 3: คำนวณ Residual Standard Error (RSE)  
$$  \text{RSE} = \sqrt{\frac{\text{RSS}}{n-2}} $$

In [ ]:
n = y.shape[0]
RSE = np.sqrt(RSS / (n - 2))
print("RSE:", RSE)

In [ ]:
# TODO: Calculate Residual Standard Error
RSE = np.sqrt()

print(RSE)

#### Exercise 4: คำนวณ Standard Error (SE)  
\begin{align*}
\text{SE}(\hat{\beta}_0) &= \text{RSE}\sqrt{\frac{1}{n}+\frac{\bar{x}^2}{\sum_{i=1}^n (x_i-\bar{x})^2}} \\
\text{SE}(\hat{\beta}_1) &= \text{RSE}\sqrt{\frac{1}{\sum_{i=1}^n (x_i-\bar{x})^2}}
\end{align*}

In [ ]:
x_mean = np.mean(X)
SE_beta_1 = RSE / np.sqrt(np.sum(np.square(X - x_mean)))
SE_beta_0 = RSE * np.sqrt((1/n) + (x_mean**2 / np.sum(np.square(X - x_mean))))

print('SE(beta_0):', SE_beta_0)
print('SE(beta_1):', SE_beta_1)

#### Exercise 5: คำนวณขอบล่าง (lower) และขอบบน (upper) ของ confidence interval
$$I_i = [\hat{\beta}_i-2\cdot\text{SE}(\hat{\beta}_i),\hat{\beta}_i+2\cdot\text{SE}(\hat{\beta}_i)]$$

In [ ]:
lower_0 = beta_0 - 2 * SE_beta_0
upper_0 = beta_0 + 2 * SE_beta_0

lower_1 = beta_1 - 2 * SE_beta_1
upper_1 = beta_1 + 2 * SE_beta_1

print(f'Confidence interval of β₀: [{lower_0}, {upper_0}]')
print(f'Confidence interval of β₁: [{lower_1}, {upper_1}]')

#### Code ข้างล่างนี้แสดงผลของค่าทั้งหมดที่เราคำนวณไปแล้ว

In [ ]:
X = auto_df['horsepower']

# Present results
results = pd.DataFrame({'feature': ['Intercept', X.name],
                        'coefficients': [beta_0,beta_1],
                        'standard_error': [SE_beta_0,SE_beta_1],
                        '[0.025': [lower_0,lower_1],
                        '0.975]': [upper_0,upper_1]})

results

## Linear Regression in `Scikit-Learn`

#### [Documentation](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html)

In [ ]:
from sklearn.linear_model import LinearRegression

reg = LinearRegression()
reg.fit(X.to_numpy().reshape(-1, 1), y.to_numpy().reshape(-1, 1))

#### ความชัน

#### ค่าตัดแกน

#### ทำนายจากตัวแปรต้น

In [ ]:
y_pred = reg.predict(X.to_numpy().reshape(-1, 1))

y_pred

## Linear Regression in `statsmodels

#### [Documentation](https://www.statsmodels.org/stable/regression.html)

#### มีสองวิธึในการ fit linear regression:

1. `statsmodels.api.sm.OLS`

In [ ]:
import statsmodels.api as sm

In [ ]:
y = auto_df['mpg']
X = auto_df['horsepower']

# add bias constant;
# without this the equation turns into y = βx
X_one = sm.add_constant(X)

X_one.head()

In [ ]:
# syntax is OLS(response, predictor)
model1 = sm.OLS(y, X_one)
model1 = model1.fit()
print(model1.summary())

In [ ]:
# Make predictions
y_pred_sm = model1.predict(X_one)

y_pred_sm.head()

2. `statsmodels.formula.api.smf.ols`

In [ ]:
import statsmodels.formula.api as smf

#syntax is ols(formula, dataset)
model2 = smf.ols('mpg ~ horsepower', auto_df)
model2 = model2.fit()
print(model2.summary())

In [ ]:
# Make predictions
y_pred_fm = model2.predict(X)

y_pred_fm.head()

In [ ]:
y_pred_fm = model2.predict(auto_df)

y_pred_fm.head()

#### Exercise 6: จงตอบคำถามต่อไปนี้

1. จากโมเดลนี้ ถ้ารถยนต์มีแรงม้า 200 hp จะมีระยะการวิ่งเท่าไหร่โดยประมาณ?
จากสมการเชิงเส้นที่ได้จากการวิเคราะห์:
\hat{y} = \beta_0 + \beta_1 x
โดยเมื่อแทนค่า horsepower = 200 เข้าไปในสมการ จะได้ระยะทางที่คาดการณ์ (mpg) ประมาณ 9.48 ไมล์ต่อแกลลอน
ซึ่งหมายความว่า ถ้ารถยนต์มีแรงม้า 200 hp จะสามารถวิ่งได้ประมาณ 9.48 ไมล์ต่อการเติมน้ำมันหนึ่งแกลลอน

2. บอกความหมายของช่วงความเชื่อมั่นของค่าคาดการณ์เมื่อ x = 200
ช่วงความเชื่อมั่น (Confidence Interval) ที่ระดับ 95% สำหรับค่าที่ทำนายเมื่อแรงม้าเป็น 200 คือ
[7.40, 11.56]
ซึ่งหมายความว่า เรามั่นใจ 95% ว่า ระยะทางจริงของรถยนต์ที่มีแรงม้า 200 hp จะอยู่ในช่วงนี้
หากสมมติฐานของโมเดล (เช่น ความสัมพันธ์เป็นเส้นตรง และการแจกแจงปกติของ error term) เป็นจริง

3. ระหว่างช่วงของสัมประสิทธิ์ β₀ กับ β₁ ช่วงไหนกว้างกว่ากัน
จากการคำนวณช่วงความเชื่อมั่นของสัมประสิทธิ์ทั้งสอง:
	•	ช่วงของ β₀ (intercept) = [46.96, 62.59] → ความกว้างประมาณ 15.63
	•	ช่วงของ β₁ (slope) = [-0.52, -0.43] → ความกว้างประมาณ 0.09

ดังนั้น ช่วงของ β₀ กว้างกว่าช่วงของ β₁ มาก
เนื่องจาก intercept (β₀) มีความไม่แน่นอนมากกว่า และได้รับผลกระทบจากค่ากลางของ X มากกว่าสโลป


#### Extra: Plotting data and regression line

In [ ]:
import matplotlib.pyplot as plt

X = auto_df['horsepower']

# Scatter plot ของ x และ y
plt.scatter(X, y)

# เส้นของ linear regression
plt.plot(X, y_pred);